<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/09_17_22_SBERT%2Bpt_biggraph_Embeddings_NN_Politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!pip install -U sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 1.8 MB 51.7 MB/s 
     |████████████████████████████████| 4.9 MB 48.2 MB/s 
     |████████████████████████████████| 1.3 MB 61.5 MB/s 
     |████████████████████████████████| 120 kB 66.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
     |████████████████████████████████| 181 kB 46.2 MB/s 
     |████████████████████████████████| 158 kB 54.2 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 157 kB 52.9 MB/s 
     |████████████████████████████████| 157 kB 51.2 MB/s 
     |████████████████████████████████| 157 kB 56.5 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 55.6 MB/s 
     |████████████████████████████████| 157 kB 58.4 MB/s 
     |█████████████████████████████

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

# Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [5]:
dataset_id = 'politifact'

In [6]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv", converters=converters)
df.head()

,Unnamed: 0,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,0,Actress Emma Stone ‘For the first time in his...,NaN,[],"[1020554564334964741, 1020817527046197248, 106...",fake,NaN,[],2911,7.976595,0,0.0
1,1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0
2,2,Friendly Fire … Charlottesville Car attacker ...,NaN,[],"[3265439004, 3250621593, 3253922920, 326691851...",fake,NaN,[],24,3.218876,0,0.0
3,3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,[],"[911971426571255810, 1036749614853103616, 1033...",fake,mightynest.com/learn/research-by-concern/dange...,[],2569,7.851661,0,0.0
4,4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...,[],0,0.000000,0,0.0


# Using the embeddings for title and text from previous experiments

In [7]:
title_embedding_file = base_dir/f"{dataset_id}_sbert_title_embeddings.npy"

if title_embedding_file.exists():
    title_embeddings = np.load(title_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    titles = df.title.tolist()
    title_embeddings = model.encode(titles, show_progress_bar=True)
    np.save(title_embedding_file, title_embeddings)

In [8]:
text_embedding_file = base_dir/f"{dataset_id}_sbert_fulltext_embeddings.npy"

if text_embedding_file.exists():
    text_embeddings = np.load(text_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    texts = (df.title + "\n" + df.text).tolist()
    text_embeddings = model.encode(texts, show_progress_bar=True)
    np.save(text_embedding_file, text_embeddings)

In [9]:
df["num_retweets"] = df.retweets.map(lambda x: len(set(x)))
df["log_num_retweets"] = np.log1p(df.num_retweets.to_numpy())
df["num_tweets"] = df.tweets.map(lambda x: len(set([t["id"] for t in x])))
df["log_num_tweets"] = np.log1p(df.num_tweets.to_numpy())

# Training - Saving the machine learning model specifications

In [10]:
import tensorflow as tf

In [11]:
def get_model(d_h=64, p_in:float=0.2, p_out:float=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Dropout(p_in), # input dropout
        tf.keras.layers.Dense(d_h, activation='relu'),
        tf.keras.layers.Dropout(p_out),  
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  optimizer='adam',metrics=['accuracy'])
    return model

In [12]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model()
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    logits = model.predict(X_test).squeeze()
    y_pred = (logits > 0.).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

### PyTorch_BigGraph embeddings

In [13]:
edim = 128
bg_embeddings = np.load(base_dir/f'{dataset_id}_pt_biggraph_article_embeddings_{edim}.npy')
idx = np.load(base_dir/f"{dataset_id}_pt_biggraph_article_idx_{edim}.npy")

In [14]:
graph_embeddings = np.zeros((text_embeddings.shape[0], edim))
graph_embeddings[idx] = bg_embeddings

In [15]:
params = {
    "epochs":200,
    "bs":128,
    # 'lr':1e-3
}

In [16]:
X = np.concatenate([
    text_embeddings, 
    df.log_num_retweets.to_numpy()[..., None], 
    df.log_num_tweets.to_numpy()[..., None],
    graph_embeddings], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 898), (894,))

In [17]:
skf = StratifiedKFold(shuffle=True, random_state=124)

Creating a group and logging all train, test data in wandb for better tracking and visualisation.

In [18]:
GROUP = f"{dataset_id}-fulltext-sbert+tw+ptbg128-mlp"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

wandb: Adding directory to artifact (/content/wandb/run-20220917_093721-neauh7i7/files/model-best)... Done. 0.1s


6/6 [==============================] - 1s 229ms/step - loss: 0.0586 - accuracy: 0.9804 - val_loss: 0.1285 - val_accuracy: 0.9330
Epoch 95/200
6/6 [==============================] - 0s 8ms/step - loss: 0.0582 - accuracy: 0.9832 - val_loss: 0.1290 - val_accuracy: 0.9274
Epoch 96/200
1/6 [====>.........................] - ETA: 0s - loss: 0.0695 - accuracy: 0.9688

wandb: Adding directory to artifact (/content/wandb/run-20220917_093721-neauh7i7/files/model-best)... Done. 0.1s


6/6 [==============================] - 1s 167ms/step - loss: 0.0652 - accuracy: 0.9804 - val_loss: 0.1278 - val_accuracy: 0.9274
Epoch 97/200
6/6 [==============================] - 0s 10ms/step - loss: 0.0573 - accuracy: 0.9874 - val_loss: 0.1280 - val_accuracy: 0.9330
Epoch 98/200
6/6 [==============================] - 0s 11ms/step - loss: 0.0655 - accuracy: 0.9818 - val_loss: 0.1289 - val_accuracy: 0.9385
Epoch 99/200
6/6 [==============================] - 0s 10ms/step - loss: 0.0648 - accuracy: 0.9804 - val_loss: 0.1309 - val_accuracy: 0.9274
Epoch 100/200
6/6 [==============================] - 0s 9ms/step - loss: 0.0514 - accuracy: 0.9860 - val_loss: 0.1347 - val_accuracy: 0.9162
Epoch 101/200
6/6 [==============================] - 0s 8ms/step - loss: 0.0656 - accuracy: 0.9804 - val_loss: 0.1359 - val_accuracy: 0.9162
Epoch 102/200
6/6 [==============================] - 0s 13ms/step - loss: 0.0548 - accuracy: 0.9804 - val_loss: 0.1386 - val_accuracy: 0.9106
Epoch 103/200
6/6 [=====

accuracy,▁▄▆▆▇▇▇▇▇▇▇▇█▇███████▇██████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▆▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▇▇█▇▇▇▇▇██▇▇▇▇▇▇██▇▇██████████████████
val_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.98741
best_epoch,95
